<a href="https://colab.research.google.com/github/naveedhsk/ai-reliability-rag/blob/main/ai_ra_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# === one-cell bootstrap ===
import os, sys, subprocess
root = "/content/ai_rmf_poc"; os.makedirs(root, exist_ok=True)
files = {
"requirements.txt": "chromadb>=0.5.3\nsentence-transformers>=3.0.1\npypdf>=4.2.0\npandas>=2.2.2\n",
"util.py": r'''import os, re
from typing import List, Tuple
from pypdf import PdfReader
def load_pdfs(data_dir: str) -> List[Tuple[str,int,str]]:
    paths, results = [], []
    for r,_,fs in os.walk(data_dir):
        for f in fs:
            if f.lower().endswith(".pdf"): paths.append(os.path.join(r,f))
    for p in sorted(paths):
        try:
            r = PdfReader(p)
            for i, page in enumerate(r.pages):
                try: text = page.extract_text() or ""
                except: text = ""
                if text.strip(): results.append((p, i+1, text))
        except Exception as e:
            print("[WARN] Failed:", p, e)
    return results
def chunk_text(t, chunk_size=1000, overlap=200):
    import re; t = re.sub(r"\s+"," ",t).strip()
    out=[]; s=0; n=len(t)
    while s<n:
        e=min(s+chunk_size,n); out.append(t[s:e])
        if e==n: break
        s=max(0,e-overlap)
    return out
''',
"ingest.py": r'''import os, argparse, uuid, chromadb
from sentence_transformers import SentenceTransformer
from util import load_pdfs, chunk_text
def main(data_dir, db_path, collection="docs", chunk_size=1000, overlap=200, batch=64):
    os.makedirs(db_path, exist_ok=True)
    client = chromadb.PersistentClient(path=db_path)
    coll = client.get_or_create_collection(name=collection)
    model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
    pages = load_pdfs(data_dir)
    if not pages: print("No PDFs in", data_dir); return
    texts=metas=ids=[]
    texts, metas, ids = [], [], []
    for (src, pg, txt) in pages:
        for i, ch in enumerate(chunk_text(txt, chunk_size, overlap)):
            texts.append(ch); metas.append({"source":src,"page":pg}); ids.append(f"{uuid.uuid4().hex}_{pg}_{i}")
            if len(texts)>=batch:
                embs = model.encode(texts, normalize_embeddings=True).tolist()
                coll.add(documents=texts, metadatas=metas, embeddings=embs, ids=ids)
                texts, metas, ids = [], [], []
    if texts:
        embs = model.encode(texts, normalize_embeddings=True).tolist()
        coll.add(documents=texts, metadatas=metas, embeddings=embs, ids=ids)
    print(f"✅ Ingest complete. Count={coll.count()} DB={db_path}")
if __name__=="__main__":
    ap=argparse.ArgumentParser()
    ap.add_argument("--data_dir", required=True)
    ap.add_argument("--db", default="./chroma_db")
    ap.add_argument("--collection", default="docs")
    ap.add_argument("--chunk_size", type=int, default=1000)
    ap.add_argument("--overlap", type=int, default=200)
    a=ap.parse_args(); main(a.data_dir,a.db,a.collection,a.chunk_size,a.overlap)
''',
"ask_cli.py": r'''import os, argparse, time, chromadb, csv
from sentence_transformers import SentenceTransformer
from datetime import datetime
def log_row(path,row):
    ex=os.path.exists(path)
    with open(path,"a",newline="") as f:
        w=csv.DictWriter(f,fieldnames=["ts","query","top_sources","latency_s","answer_len","model","usage_tokens"])
        if not ex: w.writeheader()
        w.writerow(row)
def main(db_path, collection, query, top_k):
    client=chromadb.PersistentClient(path=db_path)
    coll=client.get_or_create_collection(name=collection)
    embed=SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
    q_emb=embed.encode([query], normalize_embeddings=True).tolist()
    t0=time.time()
    res=coll.query(query_embeddings=q_emb, n_results=top_k, include=["documents","metadatas"])
    latency=time.time()-t0
    docs=res["documents"][0] if res["documents"] else []
    metas=res["metadatas"][0] if res["metadatas"] else []
    pairs=[(os.path.basename(m.get("source","")), m.get("page","?"), d) for d,m in zip(docs,metas)]
    if not pairs:
        answer="No relevant excerpts found."
        top_sources=""
    else:
        answer="Top relevant excerpts:\n" + "\n\n".join([f"- ({s}:{p}) {d[:500]}..." for s,p,d in pairs])
        top_sources=";".join([f"{s}:{p}" for s,p,_ in pairs])
    print("\n=== ANSWER ===\n", answer, "\n")
    print("=== SOURCES ==="); [print(f"- {s} (p.{p})") for s,p,_ in pairs]
    os.makedirs("./logs", exist_ok=True)
    log_row("./logs/interactions.csv", {
        "ts": datetime.utcnow().isoformat(),
        "query": query,
        "top_sources": top_sources,
        "latency_s": round(latency,3),
        "answer_len": len(answer),
        "model": "extractive",
        "usage_tokens": ""
    })
if __name__=="__main__":
    ap=argparse.ArgumentParser()
    ap.add_argument("--db", default="./chroma_db")
    ap.add_argument("--collection", default="docs")
    ap.add_argument("--q", required=True)
    ap.add_argument("--k", type=int, default=4)
    a=ap.parse_args(); main(a.db,a.collection,a.q,a.k)
''',
}
for p,c in files.items():
    with open(f"{root}/{p}","w") as f: f.write(c)
os.makedirs(f"{root}/data", exist_ok=True); os.makedirs(f"{root}/logs", exist_ok=True)
print("Installing deps…")
subprocess.run([sys.executable,"-m","pip","install","-q","-r",f"{root}/requirements.txt"], check=False)
print("✅ Project created at", root)
print("Next:")
print("1) Upload PDFs into /content/ai_rmf_poc/data (left Files panel)")
print("2) Ingest:  !python /content/ai_rmf_poc/ingest.py --data_dir /content/ai_rmf_poc/data --db /content/ai_rmf_poc/chroma_db")
print('3) Ask:     !python /content/ai_rmf_poc/ask_cli.py --db /content/ai_rmf_poc/chroma_db --q "What are the core functions in the NIST AI RMF?"')


Installing deps…
✅ Project created at /content/ai_rmf_poc
Next:
1) Upload PDFs into /content/ai_rmf_poc/data (left Files panel)
2) Ingest:  !python /content/ai_rmf_poc/ingest.py --data_dir /content/ai_rmf_poc/data --db /content/ai_rmf_poc/chroma_db
3) Ask:     !python /content/ai_rmf_poc/ask_cli.py --db /content/ai_rmf_poc/chroma_db --q "What are the core functions in the NIST AI RMF?"


# New Section

In [1]:
# If you already ran the bootstrap earlier, you can SKIP this cell.
import os, subprocess, sys, textwrap

root = "/content/ai_rmf_poc"
os.makedirs(root + "/data", exist_ok=True)
os.makedirs(root + "/logs", exist_ok=True)

# Minimal deps for ingest/ask + downloads
%pip -q install chromadb==0.5.3 sentence-transformers==3.0.1 pypdf==4.2.0 pandas==2.2.2 requests tqdm

# Write util/ingest/ask if missing
if not os.path.exists(root + "/util.py"):
    with open(root + "/util.py","w") as f:
        f.write(textwrap.dedent(r"""
        import os, re
        from typing import List, Tuple
        from pypdf import PdfReader
        def load_pdfs(data_dir: str) -> List[Tuple[str,int,str]]:
            paths, results = [], []
            for r,_,fs in os.walk(data_dir):
                for fn in fs:
                    if fn.lower().endswith(".pdf"): paths.append(os.path.join(r,fn))
            for p in sorted(paths):
                try:
                    reader = PdfReader(p)
                    for i, page in enumerate(reader.pages):
                        try: txt = page.extract_text() or ""
                        except: txt = ""
                        if txt.strip(): results.append((p, i+1, txt))
                except Exception as e:
                    print("[WARN] Failed:", p, e)
            return results
        def chunk_text(t, chunk_size=1000, overlap=200):
            t = re.sub(r"\s+"," ",t).strip()
            out=[]; s=0; n=len(t)
            while s<n:
                e=min(s+chunk_size,n); out.append(t[s:e])
                if e==n: break
                s=max(0,e-overlap)
            return out
        """))
    with open(root + "/ingest.py","w") as f:
        f.write(textwrap.dedent(r"""
        import os, argparse, uuid, chromadb
        from sentence_transformers import SentenceTransformer
        from util import load_pdfs, chunk_text
        def main(data_dir, db_path, collection="docs", chunk_size=1000, overlap=200, batch=64):
            os.makedirs(db_path, exist_ok=True)
            client = chromadb.PersistentClient(path=db_path)
            coll = client.get_or_create_collection(name=collection)
            model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
            pages = load_pdfs(data_dir)
            if not pages: print("No PDFs in", data_dir); return
            texts, metas, ids = [], [], []
            for (src, pg, txt) in pages:
                for i, ch in enumerate(chunk_text(txt, chunk_size, overlap)):
                    texts.append(ch); metas.append({"source":src,"page":pg}); ids.append(f"{uuid.uuid4().hex}_{pg}_{i}")
                    if len(texts)>=batch:
                        embs = model.encode(texts, normalize_embeddings=True).tolist()
                        coll.add(documents=texts, metadatas=metas, embeddings=embs, ids=ids)
                        texts, metas, ids = [], [], []
            if texts:
                embs = model.encode(texts, normalize_embeddings=True).tolist()
                coll.add(documents=texts, metadatas=metas, embeddings=embs, ids=ids)
            print(f"✅ Ingest complete. Count={coll.count()} | DB={db_path}")
        if __name__=="__main__":
            ap=argparse.ArgumentParser()
            ap.add_argument("--data_dir", required=True)
            ap.add_argument("--db", default="./chroma_db")
            ap.add_argument("--collection", default="docs")
            ap.add_argument("--chunk_size", type=int, default=1000)
            ap.add_argument("--overlap", type=int, default=200)
            a=ap.parse_args(); main(a.data_dir,a.db,a.collection,a.chunk_size,a.overlap)
        """))
    with open(root + "/ask_cli.py","w") as f:
        f.write(textwrap.dedent(r"""
        import os, argparse, time, chromadb, csv
        from sentence_transformers import SentenceTransformer
        from datetime import datetime
        def log_row(path,row):
            ex=os.path.exists(path)
            with open(path,"a",newline="") as f:
                w=csv.DictWriter(f,fieldnames=["ts","query","top_sources","latency_s","answer_len","model","usage_tokens"])
                if not ex: w.writeheader()
                w.writerow(row)
        def main(db_path, collection, query, top_k):
            client=chromadb.PersistentClient(path=db_path)
            coll=client.get_or_create_collection(name=collection)
            embed=SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
            q_emb=embed.encode([query], normalize_embeddings=True).tolist()
            t0=time.time()
            res=coll.query(query_embeddings=q_emb, n_results=top_k, include=["documents","metadatas"])
            latency=time.time()-t0
            docs=res["documents"][0] if res["documents"] else []
            metas=res["metadatas"][0] if res["metadatas"] else []
            pairs=[(os.path.basename(m.get("source","")), m.get("page","?"), d) for d,m in zip(docs,metas)]
            if not pairs:
                answer="No relevant excerpts found."; top_sources=""
            else:
                answer="Top relevant excerpts:\n" + "\n\n".join([f"- ({s}:{p}) {d[:500]}..." for s,p,d in pairs])
                top_sources=";".join([f"{s}:{p}" for s,p,_ in pairs])
            print("\n=== ANSWER ===\n", answer, "\n")
            print("=== SOURCES ==="); [print(f"- {s} (p.{p})") for s,p,_ in pairs]
            os.makedirs("./logs", exist_ok=True)
            log_row("./logs/interactions.csv", {
                "ts": datetime.utcnow().isoformat(),
                "query": query,
                "top_sources": top_sources,
                "latency_s": round(latency,3),
                "answer_len": len(answer),
                "model": "extractive",
                "usage_tokens": ""
            })
        if __name__=="__main__":
            ap=argparse.ArgumentParser()
            ap.add_argument("--db", default="./chroma_db")
            ap.add_argument("--collection", default="docs")
            ap.add_argument("--q", required=True)
            ap.add_argument("--k", type=int, default=4)
            a=ap.parse_args(); main(a.db,a.collection,a.q,a.k)
        """))
print("Project ready at /content/ai_rmf_poc")


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
Project ready at /content/ai_rmf_poc


In [2]:
# If you already ran the bootstrap earlier, you can SKIP this cell.
import os, subprocess, sys, textwrap

root = "/content/ai_rmf_poc"
os.makedirs(root + "/data", exist_ok=True)
os.makedirs(root + "/logs", exist_ok=True)

# Minimal deps for ingest/ask + downloads
%pip -q install chromadb==0.5.3 sentence-transformers==3.0.1 pypdf==4.2.0 pandas==2.2.2 requests tqdm

# Write util/ingest/ask if missing
if not os.path.exists(root + "/util.py"):
    with open(root + "/util.py","w") as f:
        f.write(textwrap.dedent(r"""
        import os, re
        from typing import List, Tuple
        from pypdf import PdfReader
        def load_pdfs(data_dir: str) -> List[Tuple[str,int,str]]:
            paths, results = [], []
            for r,_,fs in os.walk(data_dir):
                for fn in fs:
                    if fn.lower().endswith(".pdf"): paths.append(os.path.join(r,fn))
            for p in sorted(paths):
                try:
                    reader = PdfReader(p)
                    for i, page in enumerate(reader.pages):
                        try: txt = page.extract_text() or ""
                        except: txt = ""
                        if txt.strip(): results.append((p, i+1, txt))
                except Exception as e:
                    print("[WARN] Failed:", p, e)
            return results
        def chunk_text(t, chunk_size=1000, overlap=200):
            t = re.sub(r"\s+"," ",t).strip()
            out=[]; s=0; n=len(t)
            while s<n:
                e=min(s+chunk_size,n); out.append(t[s:e])
                if e==n: break
                s=max(0,e-overlap)
            return out
        """))
    with open(root + "/ingest.py","w") as f:
        f.write(textwrap.dedent(r"""
        import os, argparse, uuid, chromadb
        from sentence_transformers import SentenceTransformer
        from util import load_pdfs, chunk_text
        def main(data_dir, db_path, collection="docs", chunk_size=1000, overlap=200, batch=64):
            os.makedirs(db_path, exist_ok=True)
            client = chromadb.PersistentClient(path=db_path)
            coll = client.get_or_create_collection(name=collection)
            model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
            pages = load_pdfs(data_dir)
            if not pages: print("No PDFs in", data_dir); return
            texts, metas, ids = [], [], []
            for (src, pg, txt) in pages:
                for i, ch in enumerate(chunk_text(txt, chunk_size, overlap)):
                    texts.append(ch); metas.append({"source":src,"page":pg}); ids.append(f"{uuid.uuid4().hex}_{pg}_{i}")
                    if len(texts)>=batch:
                        embs = model.encode(texts, normalize_embeddings=True).tolist()
                        coll.add(documents=texts, metadatas=metas, embeddings=embs, ids=ids)
                        texts, metas, ids = [], [], []
            if texts:
                embs = model.encode(texts, normalize_embeddings=True).tolist()
                coll.add(documents=texts, metadatas=metas, embeddings=embs, ids=ids)
            print(f"✅ Ingest complete. Count={coll.count()} | DB={db_path}")
        if __name__=="__main__":
            ap=argparse.ArgumentParser()
            ap.add_argument("--data_dir", required=True)
            ap.add_argument("--db", default="./chroma_db")
            ap.add_argument("--collection", default="docs")
            ap.add_argument("--chunk_size", type=int, default=1000)
            ap.add_argument("--overlap", type=int, default=200)
            a=ap.parse_args(); main(a.data_dir,a.db,a.collection,a.chunk_size,a.overlap)
        """))
    with open(root + "/ask_cli.py","w") as f:
        f.write(textwrap.dedent(r"""
        import os, argparse, time, chromadb, csv
        from sentence_transformers import SentenceTransformer
        from datetime import datetime
        def log_row(path,row):
            ex=os.path.exists(path)
            with open(path,"a",newline="") as f:
                w=csv.DictWriter(f,fieldnames=["ts","query","top_sources","latency_s","answer_len","model","usage_tokens"])
                if not ex: w.writeheader()
                w.writerow(row)
        def main(db_path, collection, query, top_k):
            client=chromadb.PersistentClient(path=db_path)
            coll=client.get_or_create_collection(name=collection)
            embed=SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
            q_emb=embed.encode([query], normalize_embeddings=True).tolist()
            t0=time.time()
            res=coll.query(query_embeddings=q_emb, n_results=top_k, include=["documents","metadatas"])
            latency=time.time()-t0
            docs=res["documents"][0] if res["documents"] else []
            metas=res["metadatas"][0] if res["metadatas"] else []
            pairs=[(os.path.basename(m.get("source","")), m.get("page","?"), d) for d,m in zip(docs,metas)]
            if not pairs:
                answer="No relevant excerpts found."; top_sources=""
            else:
                answer="Top relevant excerpts:\n" + "\n\n".join([f"- ({s}:{p}) {d[:500]}..." for s,p,d in pairs])
                top_sources=";".join([f"{s}:{p}" for s,p,_ in pairs])
            print("\n=== ANSWER ===\n", answer, "\n")
            print("=== SOURCES ==="); [print(f"- {s} (p.{p})") for s,p,_ in pairs]
            os.makedirs("./logs", exist_ok=True)
            log_row("./logs/interactions.csv", {
                "ts": datetime.utcnow().isoformat(),
                "query": query,
                "top_sources": top_sources,
                "latency_s": round(latency,3),
                "answer_len": len(answer),
                "model": "extractive",
                "usage_tokens": ""
            })
        if __name__=="__main__":
            ap=argparse.ArgumentParser()
            ap.add_argument("--db", default="./chroma_db")
            ap.add_argument("--collection", default="docs")
            ap.add_argument("--q", required=True)
            ap.add_argument("--k", type=int, default=4)
            a=ap.parse_args(); main(a.db,a.collection,a.q,a.k)
        """))
print("Project ready at /content/ai_rmf_poc")


Project ready at /content/ai_rmf_poc


In [3]:
!python /content/ai_rmf_poc/ingest.py --data_dir /content/ai_rmf_poc/data --db /content/ai_rmf_poc/chroma_db

# Ask a couple of test questions (extractive answers + sources; no API key needed)
!python /content/ai_rmf_poc/ask_cli.py --db /content/ai_rmf_poc/chroma_db --q "What are the four functions of the AI RMF?"
!python /content/ai_rmf_poc/ask_cli.py --db /content/ai_rmf_poc/chroma_db --q "What is an AI incident and what are typical response steps?"


2025-09-05 23:26:19.622171: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1757114779.671552   10726 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1757114779.687069   10726 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1757114779.731355   10726 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1757114779.731411   10726 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1757114779.731417   10726 computation_placer.cc:177] computation placer alr

In [ ]:
%pip install -U "numpy>=2.0,<2.3"
import numpy as np; print("NumPy:", np.__version__)

# Colab usually needs a runtime restart after upgrading NumPy:
import os, sys; os.kill(os.getpid(), 9)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 91.6 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
chromadb 0.5.3 requires numpy<2.0.0,>=1.22.5, but you have numpy 2.2.6 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.2.6 which is incompatible.
tensorflow 2.19.0 requires numpy<2.2.0,>=1.26.0, but you have numpy 2.2.6 which is incompatible.


In [3]:
import os, re, requests
from urllib.parse import urlparse
from tqdm.auto import tqdm

DATA_DIR = "/content/ai_rmf_poc/data"
os.makedirs(DATA_DIR, exist_ok=True)

PDF_URLS = [
  # NIST AI & reliability–relevant
  "https://nvlpubs.nist.gov/nistpubs/ai/nist.ai.100-1.pdf",        # AI RMF 1.0
  "https://nvlpubs.nist.gov/nistpubs/ai/NIST.AI.600-1.pdf",        # GenAI Profile
  "https://nvlpubs.nist.gov/nistpubs/SpecialPublications/NIST.SP.800-61r3.pdf",  # Incident Handling r3
  "https://nvlpubs.nist.gov/nistpubs/specialpublications/nist.sp.800-61r2.pdf",  # Incident Handling r2
  "https://nvlpubs.nist.gov/nistpubs/legacy/sp/nistspecialpublication800-30r1.pdf", # Risk Assessments
  "https://nvlpubs.nist.gov/nistpubs/CSWP/NIST.CSWP.29.pdf",       # CSF 2.0
  "https://nvlpubs.nist.gov/nistpubs/SpecialPublications/NIST.SP.800-218A.pdf",  # SSDF for GenAI/Dual-use
  "https://nvlpubs.nist.gov/nistpubs/SpecialPublications/NIST.SP.800-171r2.pdf", # 800-171 rev2
  "https://nvlpubs.nist.gov/nistpubs/ai/NIST.AI.100-2e2025.pdf",   # Adversarial ML taxonomy
  "https://nvlpubs.nist.gov/nistpubs/legacy/sp/nistspecialpublication800-39.pdf"  # Managing Info Security Risk
]

def safe_name(url: str) -> str:
    path = urlparse(url).path
    name = os.path.basename(path) or "file.pdf"
    if not name.lower().endswith(".pdf"):
        name += ".pdf"
    return re.sub(r"[^A-Za-z0-9_.-]", "_", name)

def download_pdf(url: str, dest_dir: str, timeout=60):
    fn = safe_name(url)
    dest = os.path.join(dest_dir, fn)
    if os.path.exists(dest):
        return dest, "skip (exists)"
    headers = {"User-Agent": "Mozilla/5.0 (Colab Downloader)"}
    with requests.get(url, headers=headers, stream=True, timeout=timeout) as r:
        r.raise_for_status()
        # If server lies about content-type, still write bytes
        total = int(r.headers.get("content-length", "0") or 0)
        with open(dest, "wb") as f, tqdm(total=total, unit="B", unit_scale=True, desc=fn) as pbar:
            for chunk in r.iter_content(chunk_size=1024 * 256):
                if chunk:
                    f.write(chunk); pbar.update(len(chunk))
    return dest, "ok"

results = []
for u in PDF_URLS:
    try:
        path, status = download_pdf(u, DATA_DIR)
        results.append((u, status, path))
    except Exception as e:
        results.append((u, f"error: {e}", ""))

print("\nDownload summary:")
for u, status, p in results:
    print(f"- {status:15} {u}")
print(f"\nSaved to: {DATA_DIR}")


nist.ai.100-1.pdf:   0%|          | 0.00/1.95M [00:00<?, ?B/s]

NIST.AI.600-1.pdf:   0%|          | 0.00/1.17M [00:00<?, ?B/s]

NIST.SP.800-61r3.pdf:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

nist.sp.800-61r2.pdf:   0%|          | 0.00/1.74M [00:00<?, ?B/s]

nistspecialpublication800-30r1.pdf:   0%|          | 0.00/827k [00:00<?, ?B/s]

...

NIST.SP.800-218A.pdf:   0%|          | 0.00/651k [00:00<?, ?B/s]

NIST.SP.800-171r2.pdf:   0%|          | 0.00/1.54M [00:00<?, ?B/s]

NIST.AI.100-2e2025.pdf:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

nistspecialpublication800-39.pdf:   0%|          | 0.00/1.23M [00:00<?, ?B/s]


Download summary:
- ok              https://nvlpubs.nist.gov/nistpubs/ai/nist.ai.100-1.pdf
- ok              https://nvlpubs.nist.gov/nistpubs/ai/NIST.AI.600-1.pdf
- ok              https://nvlpubs.nist.gov/nistpubs/SpecialPublications/NIST.SP.800-61r3.pdf
- ok              https://nvlpubs.nist.gov/nistpubs/specialpublications/nist.sp.800-61r2.pdf
- ok              https://nvlpubs.nist.gov/nistpubs/legacy/sp/nistspecialpublication800-30r1.pdf
- ok              https://nvlpubs.nist.gov/nistpubs/CSWP/NIST.CSWP.29.pdf
- ok              https://nvlpubs.nist.gov/nistpubs/SpecialPublications/NIST.SP.800-218A.pdf
- ok              https://nvlpubs.nist.gov/nistpubs/SpecialPublications/NIST.SP.800-171r2.pdf
- ok              https://nvlpubs.nist.gov/nistpubs/ai/NIST.AI.100-2e2025.pdf
- ok              https://nvlpubs.nist.gov/nistpubs/legacy/sp/nistspecialpublication800-39.pdf

Saved to: /content/ai_rmf_poc/data


In [4]:
!python /content/ai_rmf_poc/ingest.py --data_dir /content/ai_rmf_poc/data --db /content/ai_rmf_poc/chroma_db

# Ask a couple of test questions (extractive answers + sources; no API key needed)
!python /content/ai_rmf_poc/ask_cli.py --db /content/ai_rmf_poc/chroma_db --q "What are the four functions of the AI RMF?"
!python /content/ai_rmf_poc/ask_cli.py --db /content/ai_rmf_poc/chroma_db --q "What is an AI incident and what are typical response steps?"


2025-09-05 23:34:07.232479: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1757115247.290240   12807 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1757115247.307796   12807 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1757115247.355175   12807 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1757115247.355244   12807 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1757115247.355249   12807 computation_placer.cc:177] computation placer alr